In [ ]:
import numpy as np
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from os import listdir as ops
fileNameList=[]
fileNameList=[i for i in ops("/content/drive/MyDrive/ACL text_BS-20221030T152021Z-001/ACL text_BS") if i.endswith(".txt")]
print(fileNameList)

['A94-1031.pdf.txt', 'A94-1028.pdf.txt', 'A94-1032.pdf.txt', 'A94-1027.pdf.txt', 'A94-1036.pdf.txt', 'A94-1040.pdf.txt', 'A94-1044.pdf.txt', 'A94-1035.pdf.txt', 'A94-1042.pdf.txt', 'A94-1037.pdf.txt', 'A94-1043.pdf.txt', 'A94-1045.pdf.txt', 'A94-1034.pdf.txt', 'A94-1039.pdf.txt', 'A94-1041.pdf.txt', 'A94-1046.pdf.txt', 'A94-1038.pdf.txt', 'A97-1003.pdf.txt', 'A97-1000.pdf.txt', 'A97-1002.pdf.txt', 'A97-1004.pdf.txt', 'A97-1001.pdf.txt', 'A94-1047.pdf.txt', 'A97-1005.pdf.txt', 'A97-1011.pdf.txt', 'A97-1009.pdf.txt', 'A97-1008.pdf.txt', 'A97-1006.pdf.txt', 'A97-1012.pdf.txt', 'A97-1010.pdf.txt', 'A97-1007.pdf.txt', 'A97-1020.pdf.txt', 'A97-1014.pdf.txt', 'A97-1018.pdf.txt', 'A97-1015.pdf.txt', 'A97-1021.pdf.txt', 'A97-1016.pdf.txt', 'A97-1019.pdf.txt', 'A97-1013.pdf.txt', 'A97-1017.pdf.txt', 'A97-1026.pdf.txt', 'A97-1025.pdf.txt', 'A97-1028.pdf.txt', 'A97-1024.pdf.txt', 'A97-1023.pdf.txt', 'A97-1029.pdf.txt', 'A97-1027.pdf.txt', 'A97-1022.pdf.txt', 'A97-1035.pdf.txt', 'A97-1034.pdf.txt',

In [ ]:
from PyPDF2 import PdfFileReader as fileread
def wordList(doc):
  Wordlist=[]
  pdfFileObj = open(doc, 'rb')
  pdfReader = fileread(pdfFileObj)
  pageObj = pdfReader.getPage(0)
  Wordlist=pageObj.extractText()
  return Wordlist

In [ ]:
from gensim.parsing.preprocessing import remove_stopwords
from string import punctuation as punct
updatedlist=[]
punctuations=punct

def removePuncs(wordList):
  wordList=[i.lower() for i in wordList]
  i=0
  while i < range(len(wordList)):
    if wordList[i]=='\n' or wordList[i] in punctuations:
      temp=wordList[i]
      wordList[i]=wordList[i].replace(temp,' ')
    i=i+1
  updatedlist=remove_stopwords(wordList)
  return updatedlist


In [ ]:
from collections import Counter
def termFrequencyInDoc(wordList):
  return dict(Counter(wordList).most_common())

In [ ]:
docfreq={}
def wordDocFre(dicList):
  allwords = set().union(*(doc.keys() for doc in dicList))
  word=0
  doc=0
  while word < range(len(allwords)):
    count=0
    while doc < range(len(dicList)):
      count+=1 if dicList[doc].get(word,0)!=0 else 0
      doc+=1
    docfreq[word]=count
    word=word+1
  return docfreq

In [ ]:
from math import log
def inverseDocFre(dicList,base):
  for i in range(len(dicList)):
    dicList[i]=log(len(fileNameList)+1/dicList[i],base)
  return
  

In [ ]:
def tfidf(docList):
  listOfDic=[]
  temp=None
  for file in docList:
    listOfDic.append(termFrequencyInDoc(removePunc(wordList(file))))

  #temp=pd.DataFrame(listOfDic)    
  docfreq=wordDocFre(listOfDic)
  idf=inverseDocFre(docfreq,len(docList))
  print(docfreq)
  print(idf)
  ret=[]
  ret=list(map(lambda x:dict(map(lambda kv:(kv[0],kv[1]*idf[kv[0]]),x.items())),listOfDic))
  return ret



SyntaxError: ignored

In [ ]:
temp=tfidf(fileNameList)

In [ ]:
print(temp[0])
tfidfVal=temp

In [ ]:
def query_f(query):
  query_vocab = []
  for word in query.split():
      if word not in query_vocab:
          query_vocab.append(word)
  query_wc = {} 
  for word in query_vocab:
      query_wc[word] = query.split().count(word)
  
  relevance_scores = {} 
  for id,doc in enumerate(fileNameList):
      score = 0 
      for word in query_vocab:
          score += query_wc[word] * tfidfVal[id].get(word,0)
      relevance_scores[doc] = score
  top_five=sorted(relevance_scores.items(), key=lambda kv:
                  (kv[1], kv[0]),reverse=True)[:5]
  print("\n\n") 
  print(query)
  for i in top_five:
    print(i)
  print("\n")

In [ ]:
q=[
"LDA",
"Topic modelling",
"Generative models",
"Semantic relationships between terms",
"Natural Language Processing",
"Text Mining",
"Translation model",
"Learning procedures for the lexicon",
"Semantic evaluations",
"System results and combination",]
for i in q:
  query_f(i)

### TF-IDF Weightage Sample Code
This is a sample code to give students an idea of how TF-IDF weightage model can be applied to calculate relevance score of documents.

The example is taken from **lectures**.

**P.S Do not get confused with the answers provided for TF-IDF in lecture 4-5, since the example shown had documents from d1-d5 but in actual the collection was large, hence M and k are actually unknown in that solved solution.** <br>
**In this example, M = 5 as it is assumed that the collection has 5 docs only.**

<h3 style = 'color:purple;'>Vector Space Model (TF-IDF Weightage Model)</h3>

$$ f(q,d) = sim(q,d) =  \sum_{i=1}^n x_iy_i $$ 
q = (x_1,.....,x_n) <br>
d = (y_1,.....,y_n) <br>
x_i = count of word W_i in query. <br>
y_i = TF-IDF of word W_i in doc i.e $$ y_i = C(W_i,doc) * log_2 \frac {M+1} {k} $$
M = number of documents in the collection <br>
k = document frequency for every word in corpus


In [ ]:
#lets say we have the following documents
documents = {
    "d1" : "news about",
    "d2" : "news about organic food campaign",
    "d3" : "news of presidential campaign",
    "d4" : "news of presidential campaign presidential candidate",
    "d5" : "news of organic food campaign campaign campaign campaign"
} # a dictionary with doc# as key and doc content as value

In [ ]:
#visualize the dictionary
documents

{'d1': 'news about',
 'd2': 'news about organic food campaign',
 'd3': 'news of presidential campaign',
 'd4': 'news of presidential campaign presidential candidate',
 'd5': 'news of organic food campaign campaign campaign campaign'}

In [ ]:
#create a corpus ccontaining the vocabulary of words in the documents
corpus = [] # a list that will store words of the vocabulary
for doc in documents.values(): #iterate through documents 
    for word in doc.split(): #go through each word in the current doc
        if not word in corpus: 
            corpus.append(word) #add word in corpus if not already added

In [ ]:
#visualize the corpus 
corpus

['news',
 'about',
 'organic',
 'food',
 'campaign',
 'of',
 'presidential',
 'candidate']

In [ ]:
#lets create a dictionary that will store document frequency for each word in the corpus
df_corpus = {} #document frequency for every word in corpus
for word in corpus:
    k = 0 #initial document frequency set to 0
    for doc in documents.values(): #iterate through documents
        if word in doc.split(): #check if word in doc
            k+=1 
    df_corpus[word] = k

In [ ]:
#verify the document frequency values
df_corpus

{'news': 5,
 'about': 2,
 'organic': 2,
 'food': 2,
 'campaign': 4,
 'of': 3,
 'presidential': 2,
 'candidate': 1}

In [ ]:
#since we have calculated k (document frequency) for all the words in the corpus, next step is to calculate idf
M = len(documents) #number of documents in the collection
idf_corpus = {} #inverse_document frequency for every word in corpus
for word in corpus:
    idf_corpus[word] = np.log2((M+1) / df_corpus[word]) #log_2 ((M+1)/k) i.e inverse document frequency

In [ ]:
#visualize idf values
idf_corpus

{'news': 0.2630344058337938,
 'about': 1.584962500721156,
 'organic': 1.584962500721156,
 'food': 1.584962500721156,
 'campaign': 0.5849625007211562,
 'of': 1.0,
 'presidential': 1.584962500721156,
 'candidate': 2.584962500721156}

We have successfully calculated inverse_document_frequency for all the words in the corpus. Now, using the idf values, we need to calculate tf-idf for each word with respect to a particular document.

In [ ]:
#calculating tf_idf
tf_idf_docs = {} #will store tf_idf scores for document words
for doc_id in documents.keys():
    tf_idf_docs[doc_id] = {} #initialize empty dictionary for each doc_id

In [ ]:
#visualize tf_idf initial state
tf_idf_docs

{'d1': {}, 'd2': {}, 'd3': {}, 'd4': {}, 'd5': {}}

In [ ]:
#finalizing the tf_idf calculations
for word in corpus:
    for doc_id,doc in documents.items(): #iterate through key,value pairs where key = doc_id and value = doc content
        tf_idf_docs[doc_id][word] = doc.split().count(word) * idf_corpus[word] #C(W_i,doc) * IDF(W_i) 

In [ ]:
#visualize final tf_idf scores for each doc
tf_idf_docs

{'d1': {'about': 1.584962500721156,
  'campaign': 0.0,
  'candidate': 0.0,
  'food': 0.0,
  'news': 0.2630344058337938,
  'of': 0.0,
  'organic': 0.0,
  'presidential': 0.0},
 'd2': {'about': 1.584962500721156,
  'campaign': 0.5849625007211562,
  'candidate': 0.0,
  'food': 1.584962500721156,
  'news': 0.2630344058337938,
  'of': 0.0,
  'organic': 1.584962500721156,
  'presidential': 0.0},
 'd3': {'about': 0.0,
  'campaign': 0.5849625007211562,
  'candidate': 0.0,
  'food': 0.0,
  'news': 0.2630344058337938,
  'of': 1.0,
  'organic': 0.0,
  'presidential': 1.584962500721156},
 'd4': {'about': 0.0,
  'campaign': 0.5849625007211562,
  'candidate': 2.584962500721156,
  'food': 0.0,
  'news': 0.2630344058337938,
  'of': 1.0,
  'organic': 0.0,
  'presidential': 3.169925001442312},
 'd5': {'about': 0.0,
  'campaign': 2.3398500028846247,
  'candidate': 0.0,
  'food': 1.584962500721156,
  'news': 0.2630344058337938,
  'of': 1.0,
  'organic': 1.584962500721156,
  'presidential': 0.0}}

### Querying the documents for relevance scores
Since we have calculated the term frequencies for all the documents in our collection, let us calcualte the relevance score of each document for a given query.

In [ ]:
query = "news about presidential campaign" #the query
query

'news about presidential campaign'

In [ ]:
query_vocab = [] # will store the unique words that occur in the query
for word in query.split():
    if word not in query_vocab:
        query_vocab.append(word)

In [ ]:
query_vocab # the unique words in the query

['news', 'about', 'presidential', 'campaign']

In [ ]:
query_wc = {} # a dictionary to store count of a word in the query (i.e x_i according to lecture slides terminology)
for word in query_vocab:
    query_wc[word] = query.split().count(word)

In [ ]:
query_wc # the count of each word that occurs in the query

{'about': 1, 'campaign': 1, 'news': 1, 'presidential': 1}

In [ ]:
relevance_scores = {} # a dictionary that will store the relevance score for each doc
# doc_id will be the key and relevance score the value for this dictionary
for doc_id in documents.keys():
    score = 0 #initialze the score for the doc to 0 at the start
    for word in query_vocab:
        score += query_wc[word] * tf_idf_docs[doc_id][word] # count of word in query * term_freq of the word
    relevance_scores[doc_id] = score

In [ ]:
# lets print the relevance scores for the query
print("Document Relevancy Scores\n",relevance_scores)

Document Relevancy Scores
 {'d1': 1.84799690655495, 'd2': 2.432959407276106, 'd3': 2.432959407276106, 'd4': 4.017921907997263, 'd5': 2.6028844087184186}
